In [1]:
"""
# You may need to set the BEETSDIR environment variable to point to your
# local beets configuration directory. 
"""
import os

os.environ["BEETSDIR"] = os.path.abspath("../../../local/config/beets")
os.environ["BEETSFLASKDIR"] = os.path.abspath("../../../local/config/beets-flask")

# Preview & Import Session with `beets-flask`

We can continue an import of any existing session we started by using custom serialization logic. We accomplish this by implementing our own beet session and task logic. This allows us to save the state of our import session and resume it later. 

For example we might want to generate a preview automatically and then manually trigger imports into our beets library afterwards.

## Triggering an automatic preview (i.e. fetching candidates)

Let's assume we have some music files `./local/inbox/album` for which we want to fetch the candidates.


In [2]:
from pathlib import Path

album_path = Path("../../../local/music/inbox/Rush")

For this album we can start a resumable import by creating a session state object and then calling any session with the state object. For instance triggering an preview would use the following:

In [3]:
from beets_flask.importer.session import PreviewSession, SessionState

# Create session state
session_state = SessionState(album_path)

# Create a preview session
session = PreviewSession(session_state)

# Trigger the fetch of the candidates
await session.run_async()

[INFO] beets-flask: Running PreviewSession on state<self.state.id='ed6bcdc4-5891-48aa-8f64-0f5767f4bf61'>.
[INFO] beets-flask: Completed PreviewSession on state<self.state.id='ed6bcdc4-5891-48aa-8f64-0f5767f4bf61'>.


SessionState(id='ed6bcdc4-5891-48aa-8f64-0f5767f4bf61', _task_states=[TaskState(id='ce283cf6-f192-47f1-9365-ad4a2a55ab77', task=<beets.importer.ImportTask object at 0x7f1f1a86d690>, candidate_states=[CandidateState(id='848e7f68-defb-407c-a57d-5fe8fe808362', duplicate_ids=[117], match=AlbumMatch(distance=<beets.autotag.hooks.Distance object at 0x7f1f1a367750>, info={'album': 'Rush / Typhoon', 'album_id': '2dwN4ymp8jSF0gROx5LjSx', 'artist': 'Basstripper', 'artist_id': '1tSiIyp5dxfbEaS0nZGMEl', 'artists': [], 'artists_ids': [], 'tracks': [{'title': 'Rush', 'track_id': '0d02JO9X19OH6xgqebGATC', 'release_track_id': None, 'artist': 'Basstripper', 'artist_id': '1tSiIyp5dxfbEaS0nZGMEl', 'artists': [], 'artists_ids': [], 'length': 274.3, 'index': 1, 'media': None, 'medium': 1, 'medium_index': 1, 'medium_total': 2, 'artist_sort': None, 'artists_sort': [], 'disctitle': None, 'artist_credit': None, 'artists_credit': [], 'data_source': 'Spotify', 'data_url': 'https://open.spotify.com/track/0d02JO9X

The session state now contains the all the session information which is needed to reconstruct the session.

The session state object a hierachy close to the beets internal logic:
- SessionState: Reflects the state of the import session.
- TaskState: Reflects a beetstask, but they dont have such a precise real-life meaning.
- CandidateState: Reflects a beets match (i.e. a candidate the user might choose)

In [4]:
for c in session_state.task_states[0].candidate_states:
    print(f"{c.distance} | {c.album:>30} | {c.match.info.data_source}")

0.03 |                 Rush / Typhoon | Spotify
0.61 |      Punishment / Blatherskite | MusicBrainz
0.64 |                 Cascade / Move | MusicBrainz
0.64 |          Greedy / Darkest Void | MusicBrainz
0.72 |              Hazmat / Deserted | MusicBrainz
0.73 |                 RawZ / Typhoon | MusicBrainz
0.00 |                 Rush / Typhoon | asis


## Manually fetching additional candidates

After we have fetched some candidates we might notices that the candidates do not match the ones we want. In this case we can manually trigger a new candidate fetch given an search query or an
url/id.


In [5]:
from beets_flask.importer.session import AddCandidatesSession

# Define a manual search, this is doable by
# id, artist, album (all are optional)
# if none are provided, the session will
# use the items
search_ids: list[str] = ["https://open.spotify.com/album/4GEskpIGBmvs2puHNS21Et"]
search_artist: str | None = "Metric"
search_album: str | None = "Metric - EP"

# We reuse our session_state as we want to add to the existing task, alternatiely we may use a new
# session_state

session = AddCandidatesSession(
    session_state,
    search_ids=search_ids,
    search_artist=search_artist,
    search_album=search_album,
)

# Trigger the fetch of the candidates
await session.run_async()

[INFO] beets-flask: Running AddCandidatesSession on state<self.state.id='ed6bcdc4-5891-48aa-8f64-0f5767f4bf61'>.
[INFO] beets-flask: Completed AddCandidatesSession on state<self.state.id='ed6bcdc4-5891-48aa-8f64-0f5767f4bf61'>.


SessionState(id='ed6bcdc4-5891-48aa-8f64-0f5767f4bf61', _task_states=[TaskState(id='ce283cf6-f192-47f1-9365-ad4a2a55ab77', task=<beets.importer.ImportTask object at 0x7f1f1a86d690>, candidate_states=[CandidateState(id='22569512-bc6f-46f7-80ac-50f2432bc51b', duplicate_ids=[419], match=AlbumMatch(distance=<beets.autotag.hooks.Distance object at 0x7f1f19be5ed0>, info={'album': 'Metrik - EP', 'album_id': '4GEskpIGBmvs2puHNS21Et', 'artist': 'Metrik', 'artist_id': '2NCEtX40i9lLNpTg2X5583', 'artists': [], 'artists_ids': [], 'tracks': [{'title': 'Break Of Dawn', 'track_id': '4ewgMNkhH6kNPbnfD2cGsz', 'release_track_id': None, 'artist': 'Metrik', 'artist_id': '2NCEtX40i9lLNpTg2X5583', 'artists': [], 'artists_ids': [], 'length': 264.827, 'index': 1, 'media': None, 'medium': 1, 'medium_index': 1, 'medium_total': 4, 'artist_sort': None, 'artists_sort': [], 'disctitle': None, 'artist_credit': None, 'artists_credit': [], 'data_source': 'Spotify', 'data_url': 'https://open.spotify.com/track/4ewgMNkhH6

In [6]:
for c in session_state.task_states[0].candidate_states:
    print(f"{c.distance} | {c.album:>30} | {c.match.info.data_source}")

0.69 |                    Metrik - EP | Spotify
0.03 |                 Rush / Typhoon | Spotify
0.61 |      Punishment / Blatherskite | MusicBrainz
0.64 |                 Cascade / Move | MusicBrainz
0.64 |          Greedy / Darkest Void | MusicBrainz
0.72 |              Hazmat / Deserted | MusicBrainz
0.73 |                 RawZ / Typhoon | MusicBrainz
0.00 |                 Rush / Typhoon | asis


## Continue with an import after preview

We can use this session to continue a import later. For example if we want to choose the best candidate, we could use the AutoImportSession.

In [6]:
from beets_flask.importer.session import ImportSession

import_session = ImportSession(session_state)


In [7]:
await import_session.run_async()

[INFO] beets-flask: Running ImportSession on state<self.state.id='7933eba0-eaff-4a48-b2fe-60a8c353dbbc'>.
[INFO] beets.importer: duplicate-skip /mnt/Repositories/beets-flask/local/music/inbox/Rush
[INFO] beets-flask: Completed ImportSession on state<self.state.id='7933eba0-eaff-4a48-b2fe-60a8c353dbbc'>.


SessionState(id='7933eba0-eaff-4a48-b2fe-60a8c353dbbc', _task_states=[TaskState(id='cbfd8f7d-7556-4409-8172-8ddc7c61c6c8', task=<beets.importer.ImportTask object at 0x7ff9853fdb10>, candidate_states=[CandidateState(id='ae51b898-095c-49c5-9162-aaaf25b61219', duplicate_ids=[117], match=AlbumMatch(distance=<beets.autotag.hooks.Distance object at 0x7ff986114f90>, info={'album': 'Rush / Typhoon', 'album_id': '2dwN4ymp8jSF0gROx5LjSx', 'artist': 'Basstripper', 'artist_id': '1tSiIyp5dxfbEaS0nZGMEl', 'artists': [], 'artists_ids': [], 'tracks': [{'title': 'Rush', 'track_id': '0d02JO9X19OH6xgqebGATC', 'release_track_id': None, 'artist': 'Basstripper', 'artist_id': '1tSiIyp5dxfbEaS0nZGMEl', 'artists': [], 'artists_ids': [], 'length': 274.3, 'index': 1, 'media': None, 'medium': 1, 'medium_index': 1, 'medium_total': 2, 'artist_sort': None, 'artists_sort': [], 'disctitle': None, 'artist_credit': None, 'artists_credit': [], 'data_source': 'Spotify', 'data_url': 'https://open.spotify.com/track/0d02JO9X